In [76]:
import pandas as pd
import unidecode

df = pd.read_csv('Sample dataset raw.csv')
df.head()

,indexCAT,Id,StartDate,EndDate,randompropernouna,randompropernounb,category,durationmin,maxmeasure,Startmeasure,Endmeasure,Frompixelx,Frompixely,Topixelx,Topixely,unit_ids
0,78,5ff35cc8-f81b-1se4-2wed-32444fsd67h12,2022-02-03T23:35:54.000Z,2022-02-03T23:59:54.000Z,Duck/54-Left,Cat/43S,one,2,20.20,0.975,0.607,917.098991,919.123750,913.989172,919.123750,"[6digint, 6digint, 9digint, 6digint….]"
1,321,7y1b5cc8-dbf3-5574-78cd-32107fsd67h3,2022-02-03T23:54:14.000Z,2022-02-04T00:35:54.000Z,Duck/54-Left,Cat/43S,two,5,6.50,0.000,0.320,917.563237,913.989172,919.657919,913.989172,"[593960, 425393486, 425393484, 425607514, 5867..."
2,6,f81b5cc8-d3f3-5671-7832-ps907fsd67h3,2022-02-04T00:33:43.000Z,2022-02-04T01:05:04.000Z,Duck/54-Left,Cat/43S,four,8,7.08,0.042,0.042,915.098909,919.657919,919.123750,919.657919,"[425393481, 591900, 591901, 591902, 425607519,..."
3,78,z89b5cc8-xcf3-8979-kl54-6tg07fsd67h4,2022-02-04T00:55:54.000Z,2022-02-04T02:13:44.000Z,Duck/54-Left,Cat/43S,three,12,3.43,0.567,0.021,913.989172,919.123750,917.098991,919.123750,"[597740, 597741, 598525, 598524, 598026, 59852..."
4,9098,k23b5cc8-2f7k-3546-73wd-yu107fsd67h5,2022-02-04T02:15:24.000Z,2022-02-04T03:35:54.000Z,Duck/54-Left,Cat/43S,one,56,1.23,0.000,0.050,919.657919,917.098991,917.563237,913.989172,"[811774, 804440, 804441, 804442, 804443, 80444..."


In [77]:
from datetime import datetime


def getOverlapTime(id1, id2):
    sDate1 = datetime.strptime(df.iloc[id1][2], "%Y-%m-%dT%H:%M:%S.%f%z")
    eDate1 = datetime.strptime(df.iloc[id1][3], "%Y-%m-%dT%H:%M:%S.%f%z")
    sDate2 = datetime.strptime(df.iloc[id2][2], "%Y-%m-%dT%H:%M:%S.%f%z")
    eDate2 = datetime.strptime(df.iloc[id2][3], "%Y-%m-%dT%H:%M:%S.%f%z")

    if (sDate1 < sDate2 and eDate1 < sDate2) or (sDate1 > sDate2 and eDate2 < sDate1):
        return 0

    return (min(eDate1, eDate2) - max(sDate1, sDate2)).seconds / 60




In [52]:
for i in range(5):
    print(getOverlapTime(i, i + 1))

5.666666666666667
2.183333333333333
9.166666666666666
0
80.5


In [67]:
def getSmallestDuration(id1, id2):
    sDate1 = datetime.strptime(df.iloc[id1][2], "%Y-%m-%dT%H:%M:%S.%f%z")
    eDate1 = datetime.strptime(df.iloc[id1][3], "%Y-%m-%dT%H:%M:%S.%f%z")
    sDate2 = datetime.strptime(df.iloc[id2][2], "%Y-%m-%dT%H:%M:%S.%f%z")
    eDate2 = datetime.strptime(df.iloc[id2][3], "%Y-%m-%dT%H:%M:%S.%f%z")

    if eDate1 - sDate1 < eDate2 - sDate2:
        return [id1, id2]

    return [id2, id1]

In [68]:
def getUnitOverlap(id1, id2):
    units1 = unidecode.unidecode(df.iloc[id1].unit_ids.strip()[1:-1]).replace(" ", "").split(",")
    units2 = unidecode.unidecode(df.iloc[id2].unit_ids.strip()[1:-1]).replace(" ", "").split(",")
    return len([value for value in units1 if value in units2]) / len(units1)

In [69]:
def isDuplicate(id1, id2):
    if getOverlapTime(id1, id2) >= 10 and getUnitOverlap(id1, id2) >= 0.7:
        units1 = unidecode.unidecode(df.iloc[id1].unit_ids.strip()[1:-1]).replace(" ", "").split(",")
        units2 = unidecode.unidecode(df.iloc[id2].unit_ids.strip()[1:-1]).replace(" ", "").split(",")

        if len(units1) < len(units2):
            return [id1, id2]
        elif len(units1) > len(units2):
            return [id2, id1]

        else:
            return getSmallestDuration(id1, id2)

    return None

In [70]:
duplicates = []
duplicates_original = []
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        flag = isDuplicate(i, j)
        if flag is None:
            continue

        if flag not in duplicates:
            duplicates.append(flag[0])
            duplicates_original.append(flag[1])

print(duplicates)
print(duplicates_original)

[5, 6, 13]
[4, 5, 14]


In [72]:
is_duplicates_col = []
duplicates_col = []
num_of_units_col = []
for i in range(len(df)):
    if i in duplicates:
        is_duplicates_col.append(1)

        index = duplicates.index(i)
        duplicates_col.append(duplicates_original[index]+1)

    else:
        is_duplicates_col.append(0)
        duplicates_col.append("")

    units = unidecode.unidecode(df.iloc[i].unit_ids.strip()[1:-1]).replace(" ", "").split(",")
    num_of_units_col.append(len(units))



print(is_duplicates_col)
print(duplicates_col)
print(num_of_units_col)

[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
['', '', '', '', '', 5, 6, '', '', '', '', '', '', 15, '', '']
[4, 99, 88, 87, 87, 85, 85, 84, 84, 83, 82, 80, 80, 8, 12, 16]


In [78]:
df.insert(loc=1, column='IsDuplicate', value=is_duplicates_col)
df.insert(loc=2, column='RelatedDuplicate', value=duplicates_col)
df.insert(loc=3, column='NumberOfUnits', value=num_of_units_col)


In [80]:
df

,indexCAT,IsDuplicate,RelatedDuplicate,NumberOfUnits,Id,StartDate,EndDate,randompropernouna,randompropernounb,category,durationmin,maxmeasure,Startmeasure,Endmeasure,Frompixelx,Frompixely,Topixelx,Topixely,unit_ids
0,78,0,,4,5ff35cc8-f81b-1se4-2wed-32444fsd67h12,2022-02-03T23:35:54.000Z,2022-02-03T23:59:54.000Z,Duck/54-Left,Cat/43S,one,2,20.20,0.975,0.607,917.098991,919.123750,913.989172,919.123750,"[6digint, 6digint, 9digint, 6digint….]"
1,321,0,,99,7y1b5cc8-dbf3-5574-78cd-32107fsd67h3,2022-02-03T23:54:14.000Z,2022-02-04T00:35:54.000Z,Duck/54-Left,Cat/43S,two,5,6.50,0.000,0.320,917.563237,913.989172,919.657919,913.989172,"[593960, 425393486, 425393484, 425607514, 5867..."
2,6,0,,88,f81b5cc8-d3f3-5671-7832-ps907fsd67h3,2022-02-04T00:33:43.000Z,2022-02-04T01:05:04.000Z,Duck/54-Left,Cat/43S,four,8,7.08,0.042,0.042,915.098909,919.657919,919.123750,919.657919,"[425393481, 591900, 591901, 591902, 425607519,..."
3,78,0,,87,z89b5cc8-xcf3-8979-kl54-6tg07fsd67h4,2022-02-04T00:55:54.000Z,2022-02-04T02:13:44.000Z,Duck/54-Left,Cat/43S,three,12,3.43,0.567,0.021,913.989172,919.123750,917.098991,919.123750,"[597740, 597741, 598525, 598524, 598026, 59852..."
4,9098,0,,87,k23b5cc8-2f7k-3546-73wd-yu107fsd67h5,2022-02-04T02:15:24.000Z,2022-02-04T03:35:54.000Z,Duck/54-Left,Cat/43S,one,56,1.23,0.000,0.050,919.657919,917.098991,917.563237,913.989172,"[811774, 804440, 804441, 804442, 804443, 80444..."
5,123,1,5,85,8uyb5cc8-dh78-807r-7a3d-31107fsd67h6,2022-02-04T02:07:24.000Z,2022-02-04T03:55:14.000Z,Duck/54-Left,Cat/43S,two,32,51.80,0.002,0.500,919.123750,917.563237,919.123750,919.657919,"[811774, 804440, 804441, 804442, 804443, 80444..."
6,34,1,6,85,q3db5cc8-dww3-2dr4-26d-mn107fsd67h7,2022-02-04T03:35:54.000Z,2022-02-04T04:15:54.000Z,Duck/54-Left,Cat/43S,four,4,43.16,0.050,0.452,917.098991,919.123750,913.989172,919.123750,"[811774, 804440, 804441, 804442, 804443, 80444..."
7,675,0,,84,5jfb5cc8-db55-4esx-7poj-6f107fsd67h8,2022-02-04T04:06:04.000Z,2022-02-04T04:25:06.000Z,Duck/54-Left,Cat/43S,three,2,16.08,0.765,0.000,917.563237,913.989172,919.657919,917.098991,"[592894, 592897, 592899, 592464, 596941, 59619..."
8,789,0,,84,2mgb5cc8-pk98-7065-8ild-2s107fsd67h9,2022-02-04T05:32:52.000Z,2022-02-04T06:15:24.000Z,Duck/54-Left,Cat/43S,one,67,13.20,0.542,0.002,915.098909,919.657919,919.123750,919.123750,"[596177, 596182, 596185, 1451279429, 595216, 5..."
9,445,0,,83,f368bcc83wsd-sc54-1s4d-2w107fsd67h10,2022-02-04T06:35:54.000Z,2022-02-04T07:35:54.000Z,Duck/54-Left,Cat/43S,two,54,9.08,0.607,0.050,913.989172,919.123750,913.989172,913.989172,"[807481, 492930823, 807489, 807493, 422520083,..."


In [83]:
df.to_csv("output.csv", sep=',', encoding='utf-8', index=False)